In [1]:
import json
import requests
import pandas as pd

In [2]:
with open("../../secrets.json", 'r') as fr:
    content = "".join(fr.readlines())
    API_KEY = json.loads(content)['eod_histo_data_api_key']

### Get Exchanges

In [3]:
exchanges = pd.read_json(f"https://eodhistoricaldata.com/api/exchanges-list/?api_token={API_KEY}")
exchanges.head()

,Name,Code,OperatingMIC,Country,Currency
0,USA Stocks,US,"XNAS, XNYS",USA,USD
1,London Exchange,LSE,XLON,UK,GBP
2,TSX Venture Exchange,V,XTSX,Canada,CAD
3,Toronto Exchange,TO,XTSE,Canada,CAD
4,NEO Exchange,NEO,NEOE,Canada,CAD


### Get Contracts

In [32]:
exchanges[60:]

,Name,Code,OperatingMIC,Country,Currency
60,Sao Paolo Exchange,SA,BVMF,Brazil,BRL
61,Mexican Exchange,MX,XMEX,Mexico,MXN
62,London IL,IL,XLON,UK,USD
63,Zagreb Stock Exchange,ZSE,XZAG,Croatia,HRK
64,EU Bond Virtual Exchange,EUBOND,None,Belgium,EUR
65,Indices,INDX,None,Unknown,Unknown
66,Istanbul Stock Exchange,IS,XIST,Turkey,TRY
67,Bolsa de Valores de Lima,LIM,XLIM,Peru,PEN
68,Government Bonds,GBOND,None,Unknown,Unknown
69,Borsa Italiana Certificates,ETLX,None,Italy,EUR


In [4]:
EXCHANGE_CODE = "CC"
contracts = pd.read_json(f"https://eodhistoricaldata.com/api/exchange-symbol-list/{EXCHANGE_CODE}?api_token={API_KEY}&fmt=json")
contracts

,Code,Name,Country,Exchange,Currency,Type,Isin
0,$ANRX-USD,AnRKey X,Unknown,CC,USD,Currency,NaN
1,$KING-USD,King Swap,Unknown,CC,USD,Currency,NaN
2,$TRDL-USD,Strudel Finance,Unknown,CC,USD,Currency,NaN
3,0xBTC-USD,0xBitcoin,Unknown,CC,USD,Currency,NaN
4,0xMR-USD,0xMonero,Unknown,CC,USD,Currency,NaN
...,...,...,...,...,...,...,...
3745,ZUT-USD,Zero Utility Token,Unknown,CC,USD,Currency,NaN
3746,ZXC-USD,0xcert,Unknown,CC,USD,Currency,NaN
3747,ZYD-USD,Zayedcoin,Unknown,CC,USD,Currency,NaN
3748,ZYN-USD,Zynecoin,Unknown,CC,USD,Currency,NaN


In [27]:
contracts['Type'].value_counts()

ETF             1834
Common Stock     907
ETC               56
INDEX              8
Name: Type, dtype: int64

### Get Historical Quotes

In [110]:
FROM = "2022-02-23"
TO = "2022-03-23"
TICKER = "NVDA.US"
url = f"https://eodhistoricaldata.com/api/eod/{TICKER}?api_token={API_KEY}&from={FROM}&to={TO}"
quotes = pd.read_csv(url, skipfooter=1)
quotes.head()

#### Split/Dividend Adjustment

In [100]:
quotes['k'] = quotes['Close'] / quotes['Adjusted_close']
quotes['Open'] = quotes['Open'] / quotes['k']
quotes['High'] = quotes['High'] / quotes['k']
quotes['Low'] = quotes['Low'] / quotes['k']
quotes.drop(['Close', 'k'], axis='columns', inplace=True)
quotes.rename(columns={"Adjusted_close": "Close"}, inplace=True)

In [106]:
quotes.head()

,Date,Open,High,Low,Close,Volume
0,2022-02-23,237.929202,241.457756,222.924928,223.7846,56651078
1,2022-02-24,210.069827,237.909202,208.820303,237.3894,73580062
2,2022-02-25,237.119464,242.077571,233.720762,241.4778,52886551
3,2022-02-28,239.828400,246.555831,236.979589,243.7569,47731859
4,2022-03-01,242.817293,243.676965,231.231717,234.6804,41205602


### Get Dividends And Splits

In [88]:
url = f"https://eodhistoricaldata.com/api/div/{TICKER}?api_token={API_KEY}&from={FROM}&fmt=json"
dividends = requests.get(url=url)

In [89]:
dividends.json()

[{'date': '2017-05-19',
  'declarationDate': None,
  'recordDate': '2017-05-23',
  'paymentDate': '2017-06-14',
  'period': 'Quarterly',
  'value': 0.035,
  'unadjustedValue': 0.14,
  'currency': 'USD'},
 {'date': '2017-08-22',
  'declarationDate': '2017-08-10',
  'recordDate': '2017-08-24',
  'paymentDate': '2017-09-18',
  'period': 'Quarterly',
  'value': 0.035,
  'unadjustedValue': 0.14,
  'currency': 'USD'},
 {'date': '2017-11-22',
  'declarationDate': '2017-11-09',
  'recordDate': '2017-11-24',
  'paymentDate': '2017-12-15',
  'period': 'Quarterly',
  'value': 0.0375,
  'unadjustedValue': 0.15,
  'currency': 'USD'},
 {'date': '2018-02-22',
  'declarationDate': '2018-02-08',
  'recordDate': '2018-02-23',
  'paymentDate': '2018-03-16',
  'period': 'Quarterly',
  'value': 0.0375,
  'unadjustedValue': 0.15,
  'currency': 'USD'},
 {'date': '2018-05-23',
  'declarationDate': '2018-05-10',
  'recordDate': '2018-05-24',
  'paymentDate': '2018-06-15',
  'period': 'Quarterly',
  'value': 0.

In [90]:
url = f"https://eodhistoricaldata.com/api/splits/{TICKER}?api_token={API_KEY}&from={FROM}"
splits = requests.get(url=url)

In [91]:
splits.text

'Date,"Stock Splits"\n2021-07-20,4.000000/1.000000\n49'

### Speedtest historical quotes

In [104]:
for TICKER in ["NVDA.US", "TSLA.US", "MSFT.US", "AMZN.US", "NFLX.US"]:
    url = f"https://eodhistoricaldata.com/api/eod/{TICKER}?api_token={API_KEY}&from={FROM}&to={TO}"
    quotes = requests.get(url=url)
    # print("-", end="")
    # url = f"https://eodhistoricaldata.com/api/div/{TICKER}?api_token={API_KEY}&from={FROM}&fmt=json"
    # dividends = requests.get(url=url)
    # print("-", end="")
    # url = f"https://eodhistoricaldata.com/api/splits/{TICKER}?api_token={API_KEY}&from={FROM}"
    # splits = requests.get(url=url)
    print("-")


-
-
-
-
-
